# Embedding

## Item2vec
在 Word2vec 诞生之后，Embedding 的思想迅速从自然语言处理领域扩散到几乎所有机器学习领域，推荐系统也不例外。既然 Word2vec 可以对词“序列”中的词进行 Embedding，那么对于 `用户购买序列` 中的一个商品，`用户观看序列` 中的一个电影，也应该存在相应的 Embedding 方法，这就是 Item2Vec 的由来

Item2Vec 最终目标就是获得 User Embedding 和 Item Embedding

- Item Embedding：利用用户行为序列，采用 Word2vec 思想，生成每个 Item 的 Embedding
- User Embedding：由 Item Embedding 平均或聚类得到

## Graph Embedding
Item2vec 建立在“序列”样本（用户行为序列）的基础上的。而在互联网场景下，数据对象之间更多呈现的是图结构。典型的场景是由用户行为数据生成的物品全局关系图，以及加入更多属性的物品组成的知识图谱。在面对图结构的时候，传统的序列 Embedding 方法就显得力不从心了。在这样的背景下，对图结构中间的节点进行表达的 Graph Embedding 成为了新的研究方向，并逐渐在深度学习推荐系统领域流行起来

Graph Embedding 种对图结构中的节点进行 Embedding 编码的方法，最终生成的节点 Embedding 向量一般包含图的结构信息及附近节点的局部相似性信息

### Deep Walk
它的主要思想是在由物品组成的图结构上进行随机游走，产生大量物品序列，然后将这些物品序列作为训练样本输入 Word2Vec 进行训练，最终得到物品的 Embedding

**步骤**

1. 首先，我们基于原始的用户行为序列（图 a），比如用户的购买物品序列、观看视频序列等等，来构建物品关系图（图 b）。从中，我们可以看出，因为用户 U1 先后购买了物品 A 和物品 B，所以产生了一条由 A 到 B 的有向边。如果后续产生了多条相同的有向边，则有向边的权重被加强。在将所有用户行为序列都转换成物品关系图中的边之后，全局的物品关系图就建立起来了

2. 然后，我们采用随机游走的方式随机选择起始点，重新产生物品序列（图 c）。其中，随机游走采样的次数、长度等都属于超参数，需要我们根据具体应用进行调整

3. 最后，我们将这些随机游走生成的物品序列输入（图 d）的 Word2Vec 模型，生成最终的物品 Embedding 向量

<img src="../../img/deep_walk.png" width="600" >


**跳转概率**

$$
P(v_j|v_i) =
\begin{cases}
\frac{w_{ij}}{\sum_j \ \ w_{ij}} & v_j \in \mathcal{N}_+ (v_i) \\
0 & e_{ij} \notin \mathcal{E}
\end{cases}
$$

其中 $\mathcal{N}_+ (v_i)$ 是节点 $v_i$ 所有的出边集合，$\mathcal{E}$ 是物品关系图中所有边的集合，$w_{ij}$ 是节点 $v_i$ 到节点 $v_j$ 边的权重。如果物品关系图是无向无权图，那么跳转概率将是上面公式的一个特例，即权重 $w_{ij}$ 将为常数 1，且 $\mathcal{N}_+ (v_i)$ 应是节点 $v_i$ 所有“边”的集合，而不是所有“出边”的集合

### Node2vec
在 Deep Walk 的基础上调整随机游走时的跳转概率，从而控制生成 Embedding 对同质性（Homophily）和结构性（Structural Equivalence）的倾向性，进一步把不同的 Embedding 输入推荐模型，让推荐系统学习到不同的网络结构特点

**同质性**

指的是距离上相近的节点的 Embedding 应该尽量相似。如节点 u 与其相连的节点 s1,s2,s3,s4 的 Embedding 表达应该是相似的

**结构性**

指的是结构上相似的节点的 Embedding 应该尽量相似。如节点 u 和节点 s6 都是各自局域网络的中心节点，结构上相似，其 Embedding 的表达也应该是相似的

<img src="../../img/node2vec.png" width="600" >

为了使 Graph Embedding 的结果能够表达网络的“结构性”，在随机游走的过程中，我们需要让游走的过程更倾向于 BFS（Breadth First Search，宽度优先搜索），因为 BFS 会更多地在当前节点的邻域中进行游走遍历，相当于对当前节点周边的网络结构进行一次“微观扫描”。当前节点是“局部中心节点”，还是“边缘节点”，亦或是“连接性节点”，其生成的序列包含的节点数量和顺序必然是不同的，从而让最终的 Embedding 抓取到更多结构性信息。而为了表达“同质性”，随机游走要更倾向于 DFS（Depth First Search，深度优先搜索）才行，因为 DFS 更有可能通过多次跳转，游走到远方的节点上。但无论怎样，DFS 的游走更大概率会在一个大的集团内部进行，这就使得一个集团或者社区内部节点的 Embedding 更为相似，从而更多地表达网络的“同质性”

**跳转概率**

$$
P(x|v)
\begin{cases}
\frac{\pi_{vx}}{Z} & \mathrm{if} \ (v,x) \in \mathcal{E} \\
0 & \mathrm{otherwise}
\end{cases}
$$

其中 $Z$ 为归一化常数，$\pi_{vx}$ 的定义如下

$$\pi_{vx} = \alpha_{pq}(t,x) \cdot w_{vx}$$

其中 $w_{vx}$ 是边 $vx$ 的原始权重，$\alpha_{pq}(t,x)$ 定义如下

$$
\alpha_{pq}(t,x) =
\begin{cases}
\frac{1}{p} & \mathrm{if} \ d_{tx} = 0 \\
1 & \mathrm{if} \ d_{tx} = 1 \\
\frac{1}{q} & \mathrm{if} \ d_{tx} = 2
\end{cases}
$$

其中 $d_{tx}$ 表示 $t$ 到 $x$ 的距离

<img src="../../img/node2vec2.png" width="500" >

$\alpha_{pq}(t,x)$ 中的参数 $p$ 和 $q$ 共同控制着随机游走的倾向性。参数 $p$ 被称为返回参数（Return Parameter），$p$ 越小，随机游走回节点 $t$ 的可能性越大，Node2Vec 就更注重表达网络的结构性。参数 $q$ 被称为进出参数（In-out Parameter），$q$ 越小，随机游走到远方节点的可能性越大，Node2Vec 更注重表达网络的同质性。反之，当前节点更可能在附近节点游走

## Embedding 的应用
- 直接应用。直接利用 Embedding 向量的相似性实现某些推荐系统的功能
- 预训练应用。在预先训练好物品和用户的 Embedding 之后，不直接应用，而是把这些 Embedding 向量作为特征向量的一部分，跟其余的特征向量拼接起来，作为推荐模型的输入参与训练
- End2End 应用。不预先训练 Embedding，而是把 Embedding 的训练与深度学习推荐模型结合起来，采用统一的、端到端的方式一起训练，直接得到包含 Embedding 层的推荐模型

## 局部敏感哈希
传统的 Embedding 相似度的计算方法是 Embedding 间的内积运算 ，这就意味着为了筛选某个用户的候选物品，需要对候选集合中的所有物品进行遍历。在 $k$ 维的 Embedding 空间中，物品总数为 $n$，那么遍历计算用户和物品向量相似度的时间复杂度是 $O(kn)$，在物品总数动辄达到几百万量级的推荐系统中，这样的时间复杂度是承受不了的，会导致线上模型服务过程的巨大延迟

由于用户和物品的 Embedding 同处于一个向量空间内，所以召回与用户向量最相似的物品 Embedding 的过程其实是一个在向量空间内搜索最近邻的过程。如果能够找到高维空间快速搜索最近邻点的方法，那么相似 Embedding 的快速搜索问题就迎刃而解了

**思想**

局部敏感哈希（Locality Sensitive Hashing，LSH）的基本思想是让相邻的点落入同一个“桶”，这样在进行最近邻搜索时，仅需要在一个桶内，或相邻的几个桶内的元素中进行搜索即可

在欧式空间中，将高维空间中的点映射到低维空间中，原本相近的点在低维空间中依然是相近的，而原本远离的点则有一定概率变成相近的点。利用低维空间中可以保留高维空间中相近距离的关系，就可以构造局部敏感哈希桶

**分桶**

对于 Embedding 向量来说，也可以用内积操作构建局部敏感哈希“桶”

$$h^{x,b}(\pmb{v}) = \lfloor \frac{x \cdot \pmb{v} + b}{w} \rfloor$$

其中 $w$ 是分桶宽度，$b$ 是 0 和 $w$ 间的一个均匀分布的随机变量，避免分桶边界固化

映射操作损失了部分距离信息，如果仅采用一个哈希函数进行分桶，则必然存在相近点误判的情况。有效的解决方法是采用 $m$ 个哈希函数同时进行分桶，同时掉进 $m$ 个哈希函数的同一个桶的两点，是相似点的概率将大大增加。通过分桶找到相邻点的候选集合后，就可以在有限的候选集合中通过遍历找到目标点真正的 $K$ 近邻